### Quick function to apply spatial data from one raster to another
In the LANDIS environment, all raster inputs and outputs have no spatial data associated with them, yet we often want to overlay the data in a GIS or interact with the output using geospatial operators with  vector data e.g., zonal stats, etc. Generally simple to do, with the number of mapped outputs created by LANDIS, a programmatic solution is the way to go. 

This simple function leverages the fact that intitially, LANDIS inputs are most likely associated with geospatial data, and consequently we can simply use gdal to get the spatial context for our analysis output by reading in an input file (e.g., initial communities, or ecoregions etc.).

In a simple loop, you could pass a directory structure to the function and step through all maps in a directory, for example.

In [13]:
import numpy, gdal
# Inputs
# spatialRaster -       '/path/to/georegistered/raster.tif'
# nonproojectedRaster - '/path/to/raster/you/want/georegistered/'
# outputFileName -      '/path/to/georegistered/output'

def geoAppend(spatialRaster, nonprojectedRaster, outputFileName):
    
    # Read in a raster with spatial data, and identical dimensions to a LANDIS output
    # Ideal files are rasters that are input to LANDIS and also were generated inside
    # a GIS, like your ecoregion or management unit file.
    templatedf = gdal.Open(spatialRaster)
    template = templatedf.ReadAsArray()
    
    # Read in file with no spatial data -- must be same dimensions as your template
    toAppenddf = gdal.Open(nonprojectedRaster)
    toAppend = toAppenddf.ReadAsArray()
    
    # Initialize an empty geotiff
    driver = gdal.GetDriverByName('GTiff')
    
    # Set the name and dimensions of the output
    outputRaster = driver.Create(outputFileName,
                                 toAppend.shape[1],
                                 toAppend.shape[0])
    
    # Assign the geotransform and projection from the input raster
    outputRaster.SetGeoTransform(templatedf.GetGeoTransform())
    outputRaster.SetProjection(templatedf.GetProjection())
    
    # Write the array to our gdal geotiff
    outputRaster.GetRasterBand(1).WriteArray(toAppend)
    
    # Write the output raster to disk
    outputRaster.FlushCache()